# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)

---

## İş Problemi (Business Problem)
FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.  
Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu davranış öbeklenmelerine göre gruplar oluşturulacak.

---

## Veri Seti Hikayesi
Veri seti, son alışverişlerini 2020 - 2021 yıllarında **OmniChannel** (hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

**Değişkenler:**
- `master_id`: Eşsiz müşteri numarası
- `order_channel`: Alışveriş yapılan platforma ait kanal (Android, iOS, Desktop, Mobile, Offline)
- `last_order_channel`: En son alışverişin yapıldığı kanal
- `first_order_date`: Müşterinin yaptığı ilk alışveriş tarihi
- `last_order_date`: Müşterinin yaptığı son alışveriş tarihi
- `last_order_date_online`: Müşterinin online platformda yaptığı son alışveriş tarihi
- `last_order_date_offline`: Müşterinin offline platformda yaptığı son alışveriş tarihi
- `order_num_total_ever_online`: Müşterinin online platformda yaptığı toplam alışveriş sayısı
- `order_num_total_ever_offline`: Müşterinin offline platformda yaptığı toplam alışveriş sayısı
- `customer_value_total_ever_offline`: Müşterinin offline alışverişlerinde ödediği toplam ücret
- `customer_value_total_ever_online`: Müşterinin online alışverişlerinde ödediği toplam ücret
- `interested_in_categories_12`: Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi


## Veriyi Hazırlama ve Anlama (Data Understanding)


In [2]:
import pandas as pd
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width', 1000)

df_ = pd.read_csv("data.csv")
df = df_.copy()
print(f"Veri seti boyutu: {df.shape[0]} satır, {df.shape[1]} sütun\n")
display(df.head())

print("\nİlk 10 gözlem:")
display(df.head(10))

print("\nDeğişken isimleri:")
print(df.columns.tolist())

print("\nVeri setinin boyutu:")
print(df.shape)

print("\nBetimsel istatistikler:")
display(df.describe().T)

print("\nEksik değer sayıları:")
print(df.isnull().sum())

print("\nDeğişken tipleri:")
df.info()

print("\n Toplam sipariş ve toplam harcama değişkenleri ekleniyor...")
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
display(df.head())

print("\nTarih değişkenleri datetime tipine dönüştürülüyor...")
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info()

print("\nAlışveriş kanallarına göre müşteri, sipariş ve harcama dağılımı:")
display(
    df.groupby("order_channel").agg({
        "master_id": "count",
        "order_num_total": "sum",
        "customer_value_total": "sum"
    })
)

print("\n En fazla kazanç sağlayan ilk 10 müşteri:")
display(df.sort_values("customer_value_total", ascending=False).head(10))

print("\nEn fazla sipariş veren ilk 10 müşteri:")
display(df.sort_values("order_num_total", ascending=False).head(10))

def data_prep(dataframe):
    """Veri seti üzerinde temel ön hazırlık işlemlerini yapan fonksiyon."""
    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
    return dataframe

df_prep = data_prep(df.copy())
display(df_prep.head())

Veri seti boyutu: 19945 satır, 12 sütun



,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]



İlk 10 gözlem:


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.00,2.00,150.87,49.99,[KADIN]
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.00,1.00,59.99,315.94,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.00,1.00,49.99,113.64,[COCUK]
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.00,2.00,120.48,934.21,"[ERKEK, COCUK, KADIN]"
9,1143f032-440d-11ea-8b43-000d3a38a36f,Mobile,Mobile,2019-07-30,2020-10-04,2020-10-04,2019-07-30,1.00,1.00,69.98,95.98,"[KADIN, AKTIFSPOR]"



Değişken isimleri:
['master_id', 'order_channel', 'last_order_channel', 'first_order_date', 'last_order_date', 'last_order_date_online', 'last_order_date_offline', 'order_num_total_ever_online', 'order_num_total_ever_offline', 'customer_value_total_ever_offline', 'customer_value_total_ever_online', 'interested_in_categories_12']

Veri setinin boyutu:
(19945, 12)

Betimsel istatistikler:


...


 En fazla kazanç sağlayan ilk 10 müşteri:


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00,2.00,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00,45905.10
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.00,1.00,130.49,36687.80,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.00,36818.29
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00,1.00,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00,33918.10
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.00,1.00,538.94,30688.47,"[ERKEK, KADIN, AKTIFSPOR]",11.00,31227.41
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.00,3.00,18119.14,2587.20,[AKTIFSPOR],4.00,20706.34
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00,4.00,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.00,18443.57
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00,1.00,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00,16918.57
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.00,1.00,180.73,12545.37,"[ERKEK, AKTIFSPOR]",37.00,12726.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00,1.00,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00,12282.24
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.00,2.00,64.97,12038.18,"[ERKEK, KADIN]",20.00,12103.15



En fazla sipariş veren ilk 10 müşteri:


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00,2.00,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00,45905.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00,1.00,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00,12282.24
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.00,109.00,10239.46,143.98,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.00,10383.44
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.00,87.00,8432.25,139.98,[ERKEK],88.00,8572.23
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.00,81.00,3997.55,242.81,[ERKEK],83.00,4240.36
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00,1.00,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00,33918.10
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.00,74.00,4640.77,543.28,"[KADIN, AKTIFSPOR]",77.00,5184.05
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.00,3.00,292.93,5004.95,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.00,5297.88
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00,1.00,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00,16918.57
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00,4.00,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.00,18443.57


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN],5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.00,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]",5.00,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.00,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR],2.00,209.98


## RFM Metriklerinin Hesaplanması

In [5]:

print(" Veri setindeki en son alışveriş tarihi:")
print(df["last_order_date"].max())  # 2021-05-30

analysis_date = dt.datetime(2021, 6, 1)
print(f"\n Analiz tarihi: {analysis_date}")

print("\n RFM DataFrame oluşturuluyor")

rfm = pd.DataFrame()
rfm["customer_id"] = df["master_id"]
rfm["recency"] = (analysis_date - df["last_order_date"]).dt.days  
rfm["frequency"] = df["order_num_total"]
rfm["monetary"] = df["customer_value_total"]

print(f"RFM DataFrame boyutu: {rfm.shape[0]} satır, {rfm.shape[1]} sütun\n")
display(rfm.head())


 Veri setindeki en son alışveriş tarihi:
2021-05-30 00:00:00

 Analiz tarihi: 2021-06-01 00:00:00

 RFM DataFrame oluşturuluyor
RFM DataFrame boyutu: 19945 satır, 4 sütun



,customer_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105,21.00,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,186,5.00,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,135,2.00,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86,2.00,209.98


## RF ve RFM Skorlarının Hesaplanması

In [8]:
print("R, F ve M metrikleri 1-5 arası skorlara çevriliyor...")
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

print("İlk 5 müşteri için RF ve M skorları:")
display(rfm.head())

rfm["RF_SCORE"] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)
print(" RF_SCORE oluşturuldu. İlk 5 müşteri:")
display(rfm[["customer_id","recency_score","frequency_score","RF_SCORE"]].head())


rfm["RFM_SCORE"] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str)
print("RFM_SCORE oluşturuldu. İlk 5 müşteri:")
display(rfm[["customer_id","recency_score","frequency_score","monetary_score","RFM_SCORE"]].head())


R, F ve M metrikleri 1-5 arası skorlara çevriliyor...
İlk 5 müşteri için RF ve M skorları:


,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00,939.37,3,4,4
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105,21.00,2013.55,3,5,5
2,69b69676-1a40-11ea-941b-000d3a38a36f,186,5.00,585.32,2,4,3
3,1854e56c-491f-11eb-806e-000d3a38a36f,135,2.00,121.97,3,1,1
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86,2.00,209.98,3,1,1


 RF_SCORE oluşturuldu. İlk 5 müşteri:


,customer_id,recency_score,frequency_score,RF_SCORE
0,cc294636-19f0-11eb-8d74-000d3a38a36f,3,4,34
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,3,5,35
2,69b69676-1a40-11ea-941b-000d3a38a36f,2,4,24
3,1854e56c-491f-11eb-806e-000d3a38a36f,3,1,31
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,3,1,31


RFM_SCORE oluşturuldu. İlk 5 müşteri:


,customer_id,recency_score,frequency_score,monetary_score,RFM_SCORE
0,cc294636-19f0-11eb-8d74-000d3a38a36f,3,4,4,344
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,3,5,5,355
2,69b69676-1a40-11ea-941b-000d3a38a36f,2,4,3,243
3,1854e56c-491f-11eb-806e-000d3a38a36f,3,1,1,311
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,3,1,1,311


## RF Skorlarının Segment Olarak Tanımlanması

In [12]:
print("RF_SCORE segmentasyonu yapılıyor")

seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

print("Segmentler oluşturuldu. İlk 10 müşteri ve segmentleri:")
display(rfm[['customer_id', 'RF_SCORE', 'segment']].head(10))


RF_SCORE segmentasyonu yapılıyor
Segmentler oluşturuldu. İlk 10 müşteri ve segmentleri:


,customer_id,RF_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,24,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,31,about_to_sleep
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,42,potential_loyalists
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,23,at_Risk
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,11,hibernating
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,34,loyal_customers
9,1143f032-440d-11ea-8b43-000d3a38a36f,11,hibernating


## Hedef Müşteri Profillerinin Belirlenmesi ve CSV Kaydı

In [22]:
print("Segmentlerin recency, frequnecy ve monetary ortalamaları")
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])


print("Hedef müşteri profilleri oluşturuluyor...")

#1) Yeni kadın ayakkabı markası hedefi

target_segments = ["champions", "loyal_customers"]
target_customers = rfm[rfm["segment"].isin(target_segments)]["customer_id"]

#2)  Kategori filtresi ile gerçek müşteri id'lerini seçme
cust_ids_new_brand = df[
    (df["master_id"].isin(target_customers)) &
    (df["interested_in_categories_12"].str.contains("KADIN"))
]["master_id"]

#3) CSV kaydı
cust_ids_new_brand.to_csv("yeni_marka_hedef_müşteri_id.csv", index=False)
print(f"Yeni marka hedef müşteri sayısı: {cust_ids_new_brand.shape[0]}")
display(cust_ids_new_brand.head())

#1) Erkek ve Çocuk ürünlerinde indirim hedefi
#2) Uzun süredir alışveriş yapmayan, geçmişte iyi müşteriler ve yeni gelenler
target_segments_discount = ["cant_loose", "hibernating", "new_customers"]
target_customers_discount = rfm[rfm["segment"].isin(target_segments_discount)]["customer_id"]

#3) Kategori filtresi ile gerçek müşteri id'lerini seçme
cust_ids_discount = df[
    (df["master_id"].isin(target_customers_discount)) &
    ((df["interested_in_categories_12"].str.contains("ERKEK")) |
     (df["interested_in_categories_12"].str.contains("COCUK")))
]["master_id"]

#4) CSV kaydı
cust_ids_discount.to_csv("indirim_hedef_müşteri_ids.csv", index=False)
print(f"İndirim hedef müşteri sayısı: {cust_ids_discount.shape[0]}")
display(cust_ids_discount.head())


Segmentlerin recency, frequnecy ve monetary ortalamaları
Hedef müşteri profilleri oluşturuluyor...
Yeni marka hedef müşteri sayısı: 2497


0     cc294636-19f0-11eb-8d74-000d3a38a36f
1     f431bd5a-ab7b-11e9-a2fc-000d3a38a36f
8     cfbda69e-5b4f-11ea-aca7-000d3a38a36f
11    c2e15af2-9eed-11e9-9897-000d3a38a36f
33    fb840306-1219-11ea-a001-000d3a38a36f
Name: master_id, dtype: object

İndirim hedef müşteri sayısı: 2771


7     3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f
10    ae608ece-c9d8-11ea-a31e-000d3a38a36f
15    13ed97a4-b167-11e9-89fa-000d3a38a36f
19    2730793e-3908-11ea-85d6-000d3a38a36f
21    7b289956-d691-11e9-93bc-000d3a38a36f
Name: master_id, dtype: object


## BONUS Hedef Örneği: Yeni Premium Kadın Çanta Koleksiyonu

### Hedef Tanımı

FLO, portföyüne yüksek fiyat segmentinde yeni bir **kadın çanta koleksiyonu** eklemektedir. Bu koleksiyon, hem kalite hem de tasarım açısından markanın üst segment müşterilerine hitap etmektedir.  

Bu tanıtım kampanyasının başarılı olabilmesi için hedef müşteri profili aşağıdaki kriterleri sağlamalıdır:

- **Sadık müşteriler:** Daha önce FLO'dan düzenli olarak alışveriş yapmış ve yüksek harcama potansiyeline sahip müşteriler.  
- **Kadın ürünlerine ilgi duyan müşteriler:** Koleksiyonun satış başarısı, doğru hedef kitleye ulaşmakla doğrudan ilişkilidir.  

Bu nedenle kampanya için **champions** ve **loyal_customers** segmentindeki kadın kategorisiyle ilgilenen müşteriler özel olarak hedeflenecek. Kampanya kapsamında bu müşterilere özel e-posta, SMS ve uygulama bildirimleri ile koleksiyon tanıtımı yapılacaktır.  

Amaç: Yüksek fiyatlı ürünlerin satışını artırmak ve marka sadakatini güçlendirmek.


In [29]:
target_segments_premium = ["champions", "loyal_customers"]
target_customers_premium = rfm[rfm["segment"].isin(target_segments_premium)]["customer_id"]

cust_ids_premium = df[
    (df["master_id"].isin(target_customers_premium)) &
    (df["interested_in_categories_12"].str.contains("KADIN"))
]["master_id"]

cust_ids_premium.to_csv("premium_kadin_hedef_müşteri_ids.csv", index=False)
print(f"Premium kadın çanta koleksiyonu için hedef müşteri sayısı: {cust_ids_premium.shape[0]}")
display(cust_ids_premium.head())


Premium kadın çanta koleksiyonu için hedef müşteri sayısı: 2497


0     cc294636-19f0-11eb-8d74-000d3a38a36f
1     f431bd5a-ab7b-11e9-a2fc-000d3a38a36f
8     cfbda69e-5b4f-11ea-aca7-000d3a38a36f
11    c2e15af2-9eed-11e9-9897-000d3a38a36f
33    fb840306-1219-11ea-a001-000d3a38a36f
Name: master_id, dtype: object